# Digital Elevation Map Derivatives

In [36]:
from glob import glob
import sys
import pyhere

sys.path.insert(0, "../../src")
from run_mp import *
from nfi_wrangling import *

In [5]:
# List all ascii files of the digital elevation model
all_files = glob.glob("../../data/raw/dem/*.asc")
all_files

AttributeError: 'function' object has no attribute 'glob'

In [45]:
pd.read_csv(
    "../../data/raw/dem_france/all_files/BDALTIV2_25M_FXX_0100_6800_MNT_LAMB93_IGN69.asc",
    skiprows=6,
    header=None,
    delim_whitespace=True,
)

# data / raw / dem_france / all_files / BDALTIV2_25M_FXX_0075_6850_MNT_LAMB93_IGN69.asc.aux.xml

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
1,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
4,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
996,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
997,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
998,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [51]:
import glob
import rasterio
from rasterio.merge import merge
from rasterio.crs import CRS

# List all .asc files in the directory
asc_files = glob.glob("../../data/raw/dem_france/all_files/*.asc")

# List for the data
src_files_to_mosaic = []

for file in asc_files:
    src = rasterio.open(file)
    src_files_to_mosaic.append(src)

# Merge function returns a single mosaic array and the transformation info
mosaic, out_trans = merge(src_files_to_mosaic)

# Copy the metadata
out_meta = src.meta.copy()

# Update the metadata
out_meta.update(
    {
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": CRS.from_epsg(2154).to_string(),
    }
)

# Write the mosaic raster to disk
with rasterio.open("combined_dem_original.tif", "w", **out_meta) as dest:
    dest.write(mosaic)

In [52]:
import folium
import rasterio
from rasterio.warp import transform_bounds
from rasterio.enums import Resampling

# Open the .tif file
with rasterio.open("combined_dem_original.tif") as src:
    # Resample the raster to a lower resolution for faster rendering
    data = src.read(
        out_shape=(src.count, int(src.height / 4), int(src.width / 4)),
        resampling=Resampling.bilinear,
    )

    # Scale the data values to 8 bit (0 - 255) for ImageOverlay
    scaled_data = data / data.max() * 255
    scaled_data = scaled_data.astype("uint8")

    # Get the bounds of the raster
    bounds = transform_bounds(src.crs, "EPSG:4326", *src.bounds)

# Create a folium map
m = folium.Map(
    location=[(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2], zoom_start=10
)

# Add the image to the map
img = folium.raster_layers.ImageOverlay(
    image=scaled_data[0],
    bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    opacity=0.6,
    interactive=True,
    cross_origin=False,
    zindex=1,
)

img.add_to(m)

# Display the map
m